# Pipeline para video analítico - Detección de objetos

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

from videoanalytics.pipeline import process_pipeline, Source, Sink

Inicialización de Tensorflow (requerida para detector y extractor de features).

In [2]:
# comment out below line to enable tensorflow logging outputs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## Definición y ejecución de pipelines

Definición de grafos y ejecución.

In [30]:
import networkx as nx
from videoanalytics.pipeline.sources import VideoReader
from videoanalytics.pipeline.sinks import VideoWriter
from videoanalytics.pipeline.sinks.yolo4_detector import YOLOv4Detector
from videoanalytics.pipeline.sinks.obj_detector import DetectionsAnnotator, DetectionsCSVWriter, ObjectDetectorCSV

### 1. Pipeline para generar detecciones y guardar en CSV

In [24]:
# Input
INPUT_VIDEO = "../data/video/barco.mp4"
START_FRAME = 0
MAX_FRAMES = None

# Object Detector
DETECTOR_WEIGHTS_FILENAME = "../checkpoints/yolov4-416"

# Detections Annotator
DETECTOR_CLASSES_FILENAME = "../data/classes/coco.names"

# Output
OUTPUT_VIDEO = "../outputs/prueba.mp4"

# 1. Crear el contexto en el que los bloques comparten información
context = {}

# 2. Instanciar la pipeline (grafo dirigido)
pipeline = nx.DiGraph()

# 3. Agregar bloques
pipeline.add_nodes_from([
    ( "input", {
      "component": VideoReader(context,
                               video_path=INPUT_VIDEO,
                               start_frame=START_FRAME,
                               max_frames=MAX_FRAMES)
    }),
    
    ( "detector", {
        "component": YOLOv4Detector(context,weights_filename=DETECTOR_WEIGHTS_FILENAME)
    }),
    
    ( "detector2csv", {
        "component": DetectionsCSVWriter(context)
    }),    
    
    ( "detector-annot", {
        "component": DetectionsAnnotator(context,class_names_filename=DETECTOR_CLASSES_FILENAME)
    }),    
        
    ( "writer", {
        "component": VideoWriter(context,filename=OUTPUT_VIDEO)
    })
])

# 4. Definir conexiones
pipeline.add_edges_from([
    ("input", "detector"), 
    ("detector", "detector2csv"),
    ("detector", "detector-annot"), 
    ("detector-annot", "writer")
])

# 5. Eliminar bloques aislados
pipeline.remove_nodes_from(list(nx.isolates(pipeline)))

# 6. Procesar el pipeline
process_pipeline(pipeline,context)

Start frame: 0
Total frames frame: 18205


  0%|          | 0/18205 [00:00<?, ?it/s]

Initializing pipeline
Sequence: ['input', 'detector', 'detector-annot', 'writer', 'detector2csv']
Processing pipeline
Shutting down pipeline
Shutting down VideoWriter. Video saved to ../outputs/prueba.mp4


### 2. Pipeline que usa detecciones precalculadas

In [29]:
# Input
INPUT_VIDEO = "../data/video/barco.mp4"
START_FRAME = 0
MAX_FRAMES = 100

# Detector
CSV_DETECTIONS_FILENAME = "detections.csv"

# Detections Annotator
DETECTOR_CLASSES_FILENAME = "../data/classes/coco.names"

# Output
OUTPUT_VIDEO = "../outputs/prueba.mp4"

# 1. Crear el contexto en el que los bloques comparten información
context = {}

# 2. Instanciar la pipeline (grafo dirigido)
pipeline = nx.DiGraph()

# 3. Agregar bloques
pipeline.add_nodes_from([
    ( "input", {
      "component": VideoReader(context,
                                 video_path=INPUT_VIDEO,
                                 start_frame=START_FRAME,
                                 max_frames=MAX_FRAMES)
    }),
    
    ( "detector", {
      "component": ObjectDetectorCSV(context,CSV_DETECTIONS_FILENAME)
    }),
    
    ( "detector-annot", {
      "component": DetectionsAnnotator(context,class_names_filename=DETECTOR_CLASSES_FILENAME)
    }),    
    
    ( "writer", {
      "component": VideoWriter(context,filename=OUTPUT_VIDEO)
    })
])

# 4. Definir conexiones
pipeline.add_edges_from([
    ("input", "detector"), 
    ("detector", "detector-annot"), 
    ("detector-annot", "writer")
])

# 5. Eliminar bloques aislados
pipeline.remove_nodes_from(list(nx.isolates(pipeline)))

# 6. Procesar
process_pipeline(pipeline,context)

Start frame: 0
Total frames frame: 100


  0%|          | 0/100 [00:00<?, ?it/s]

Initializing pipeline
Sequence: ['input', 'detector', 'detector-annot', 'writer']
Processing pipeline
Shutting down pipeline
Shutting down VideoWriter. Video saved to ../outputs/prueba.mp4


Cierre de sesión de tensorflow.

In [25]:
session.close()